NOTES:

1. Concatenation of drive data takes a really long time.
2. Although I labeled temps as complete. We should still check whether there is anything inside AERO temps folder that is not in 'verzamelblad.xlsx'

Importing Libraries

In [1]:

import pandas as pd
import numpy as np
import glob
import os

Defining Functions

In [2]:
def csv_to_df_concatenator(list_of_paths, axis=0, svrm_drive=False, sep=","):
    '''
    given a list of paths, it will load all those CSV files and concatenate them 
    based on column names and axis if given 0 or 1.
    kwargs may also be passed to use when reading the files.
    '''
    dfs = []
    skiprows = [0, 1, 2, 4, 5] if svrm_drive else None
    
    for csv_file in list_of_paths:
        for chunk in pd.read_csv(csv_file, skiprows=skiprows, chunksize=500000, sep=sep):
            dfs.append(chunk)
    
    return pd.concat(dfs, axis=axis, ignore_index=True)
    
def df_to_feather_compatible(df):
    '''
    checks for mixed type columns and converts them to string columns, 
    since feather can't handle mixed type columns.
    '''
    for col in df.columns:
        if df[col].dtype == "object":
            dtype = pd.api.types.infer_dtype(df[col], skipna=True)
            if dtype in ("mixed", "mixed-integer", "mixed-integer-float"):
                df[col] = df[col].astype(str)
    return df

Handling Exceptions

In [3]:
temps_sheets_exceptions = ['Blad2', 'Blad3']

Reading Datasets

In [4]:
temps_xl = pd.ExcelFile('verzamelblad.xlsx')
SVRM2_drive_csv_files = glob.glob(os.path.join('SVRM2 Drive data', "*.csv"))
SVRM3_drive_csv_files = glob.glob(os.path.join('SVRM3 Drive data', "*.csv"))
SVRM3_ewon_csv_files = glob.glob(os.path.join('SVRM3 Ewon data', "*.csv"))
SVRM4_ewon_csv_files = glob.glob(os.path.join('SVRM4 Ewon data', "*.csv"))

In [5]:
temps_sheets = temps_xl.sheet_names[1:]
for sheet in temps_sheets:
    if sheet in temps_sheets_exceptions:
        small_df = pd.read_excel('verzamelblad.xlsx', sheet)
    else:
        small_df = pd.read_excel('verzamelblad.xlsx', sheet, skiprows=[0,1,2])
    if sheet == temps_sheets[0]:
        temps_df = small_df
    else:
        temps_df = pd.concat((temps_df, small_df))

In [6]:
temps_df

,TimeInt,TimeStr,Unnamed: 2,33-1-Olie,34-2-V-O-O,35-3-V-O-B,36-4-V-B-O,37-5-V-B-B,38-6-WIS-O,39-7-WIS-B,40-8-OMG,auto,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1756263602,2025-08-27 03:00:02,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,1756315138,2025-08-27 17:18:58,2,0.0,0.0,0.0,31.1,31.4,33.0,33.5,32.8,0,NaN,NaN,NaN,NaN,NaN
2,1756315139,2025-08-27 17:18:59,3,36.4,30.7,30.4,31.2,31.4,32.8,33.5,33.0,0,NaN,NaN,NaN,NaN,NaN
3,1756315140,2025-08-27 17:19:00,4,36.3,30.8,30.6,31.3,31.2,32.9,33.3,33.0,0,NaN,NaN,NaN,NaN,NaN
4,1756315141,2025-08-27 17:19:01,5,36.4,30.8,30.6,31.1,31.4,33.0,33.4,32.8,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18370,1757386775,2025-09-09 02:59:35,18371,35.2,28.7,28.3,27.8,27.9,33.8,30.3,32.4,Undef,NaN,NaN,NaN,NaN,NaN
18371,1757386780,2025-09-09 02:59:40,18372,35.3,28.7,28.3,27.8,27.9,33.9,30.3,32.2,Undef,NaN,NaN,NaN,NaN,NaN
18372,1757386785,2025-09-09 02:59:45,18373,35.3,28.6,28.4,27.7,27.8,33.8,30.5,32.3,Undef,NaN,NaN,NaN,NaN,NaN
18373,1757386790,2025-09-09 02:59:50,18374,35.3,28.9,28.4,27.9,27.9,34,30.3,32.2,Undef,NaN,NaN,NaN,NaN,NaN


In [7]:
timestamp_dupes = temps_df[temps_df.duplicated(subset=['TimeStr'])]
print(f"Number of duplicate timestamps: {len(timestamp_dupes)}")

Number of duplicate timestamps: 164180


In [8]:
# SVRM2_drive_df = csv_to_df_concatenator(SVRM2_drive_csv_files, axis=0, svrm_drive=True)

In [9]:
# SVRM3_drive_df = csv_to_df_concatenator(SVRM3_drive_csv_files, axis=0, svrm_drive=True)

In [10]:
# SVRM3_drive_df.i

In [11]:
SVRM3_ewon_df = csv_to_df_concatenator(SVRM3_ewon_csv_files, sep=';')

C:\Users\ravin\AppData\Local\Temp\ipykernel_1860\970935797.py:11: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,47) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, skiprows=skiprows, chunksize=500000, sep=sep):
C:\Users\ravin\AppData\Local\Temp\ipykernel_1860\970935797.py:11: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,47) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, skiprows=skiprows, chunksize=500000, sep=sep):
C:\Users\ravin\AppData\Local\Temp\ipykernel_1860\970935797.py:11: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,47) have mixed types. Specify dtype option on import or set low_memory=False.
  fo

In [12]:
SVRM4_ewon_df = csv_to_df_concatenator(SVRM4_ewon_csv_files, sep=';')

C:\Users\ravin\AppData\Local\Temp\ipykernel_1860\970935797.py:11: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,27,28,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, skiprows=skiprows, chunksize=500000, sep=sep):
C:\Users\ravin\AppData\Local\Temp\ipykernel_1860\970935797.py:11: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,27,28,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, skiprows=skiprows, chunksize=500000, sep=sep):
C:\Users\ravin\AppData\Local\Temp\ipykernel_1860\970935797.py:11: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,27,28,29,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, skiprows=skiprows, chunksize=500000, sep=sep):
C:\Users\ravin\AppData\Local\Te

## Export DataFrames

In [13]:
temps_df = df_to_feather_compatible(temps_df)
SVRM2_drive_df = df_to_feather_compatible(SVRM2_drive_df)

NameError: name 'SVRM2_drive_df' is not defined

In [ ]:
SVRM3_drive_df = df_to_feather_compatible(SVRM3_drive_df)

In [14]:
SVRM3_ewon_df = df_to_feather_compatible(SVRM3_ewon_df)
SVRM4_ewon_df = df_to_feather_compatible(SVRM4_ewon_df)

In [15]:
temps_df.to_feather('Temps_complete.feather')

In [ ]:
SVRM2_drive_df.to_feather('SVRM2_drive_complete.feather')
SVRM3_drive_df.to_feather('SVRM3_drive_complete.feather')

In [16]:
SVRM3_ewon_df.to_feather('SVRM3_ewon_complete.feather')
SVRM4_ewon_df.to_feather('SVRM4_ewon_complete.feather')